# Extração do vetor de características
## Tokenizaçao e contagem de palavras

In [90]:
# importaçÕes necessárias
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import nltk

import html
from bs4 import BeautifulSoup
import pandas as pd
from sklearn import tree, naive_bayes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import cross_val_predict

In [ ]:
# instalar stemmer português no nltk
nltk.download('rslp')

In [ ]:
# função para ler os arquivos e retornar em uma lista
def retorna_conteudo(pasta):
    lista_conteudos = []
    for arquivo in os.listdir(os.getcwd()+'/'+pasta):
        with open(pasta+'/'+arquivo, 'r', encoding="utf8") as arq:
            lista_conteudos.append(arq.read())
    return lista_conteudos

In [ ]:
lista_conteudos = retorna_conteudo('wikis_text')

In [ ]:
# extrair contagem de palavras com a configuração padrão
# extrair ocorrência binária
contagem = TfidfVectorizer()

vetor = contagem.fit_transform(lista_conteudos)
#nomes das características
print(contagem.get_feature_names())
# vetor de caractrísticas
print(vetor.toarray())

In [ ]:
# extrair ocorrência binária
bo = TfidfVectorizer(binary=True, norm=None, use_idf=False)

vetor = bo.fit_transform(lista_conteudos)
#nomes das características
print(bo.get_feature_names())
# vetor de caractrísticas
print(vetor.toarray())

In [ ]:
# extrair com ocorrência dos termos
to = TfidfVectorizer(binary=False, norm=None, use_idf=False)

vetor = to.fit_transform(lista_conteudos)
#nomes das características
print(to.get_feature_names())
# vetor de caractrísticas
print(vetor.toarray())

In [ ]:
# extrair com tf
tf = TfidfVectorizer(binary=False,norm='l1', use_idf=False)

vetor = tf.fit_transform(lista_conteudos)
#nomes das características
print(tf.get_feature_names())
# vetor de caractrísticas
print(vetor.toarray())

In [ ]:
# extrair com tf.idf
tfidf = TfidfVectorizer(binary=False,norm='l1', use_idf=True)

vetor = tfidf.fit_transform(lista_conteudos)
#nomes das características
print(tfidf.get_feature_names())
# vetor de caractrísticas
print(vetor.toarray())

# Processamento linguístico

## Remover o que não é palavra (considera somente termos com 3 ou mais letras

In [ ]:
# token pattern: termos com letras, com 3 ou mias caracteres
tfidf = TfidfVectorizer(binary=False,norm='l1', use_idf=True, token_pattern=r'\b[a-zA-Z]{3,}\b')
vetor = tfidf.fit_transform(lista_conteudos)
#nomes das características
print(tfidf.get_feature_names())
# vetor de caractrísticas
print(vetor.toarray())

## remover stopwords

In [12]:
# ler arquivo com stopwords e transforma em uma lista
stopwords = []
with open('stopwords.txt','r', encoding='utf-8') as s:
    stopwords = s.read().split('\n')

In [ ]:
tfidf = TfidfVectorizer(binary=False,norm='l1', use_idf=True, token_pattern=r'\b[a-zA-Z]{3,}\b', stop_words=stopwords)
vetor = tfidf.fit_transform(lista_conteudos)
#nomes das características
print(tfidf.get_feature_names())
# vetor de caractrísticas
print(vetor.toarray())

## executar stemmer

In [11]:
# método para fazer o stemmer
def stemmer(doc):
    stemmer = nltk.stem.RSLPStemmer()
    return ' '.join([stemmer.stem(t) for t in nltk.word_tokenize(doc)])

In [ ]:
tfidf = TfidfVectorizer(binary=False,norm='l1', use_idf=True, token_pattern=r'\b[a-zA-Z]{3,}\b', stop_words=stopwords, preprocessor=stemmer)
vetor = tfidf.fit_transform(lista_conteudos)
#nomes das características
print(tfidf.get_feature_names())
# vetor de caractrísticas
print(vetor.toarray())

# Classificação

In [87]:
def extrai_texto(conteudo, filtro) :
    bsObj =  BeautifulSoup(conteudo_html, "html.parser")
    for remove in bsObj(["script", "style"]): remove.extract()
    return bsObj(attrs=filtro)[0].get_text(separator=" ")

map_content = {'agricultura':{'class':'visualiza-noticia'},
               'cti':{'id':'layout-column_column-2'}, 
               'cultura':{'class':'journal-content-article'}, 
               'esportes':{'class':'item-page'}
              }
from random import randint
# abrir arquivos
textos = []
categorias = []
for pasta in os.listdir('noticias'):    
    for arquivo in os.listdir('noticias/'+pasta):
        with open('noticias/'+pasta+'/'+arquivo, 'r', encoding="ansi") as arq:
           # print(arquivo)
            conteudo_html = html.unescape(arq.read())
            conteudo = extrai_texto(conteudo_html, map_content[pasta])
            textos.append(conteudo)
            categorias.append(pasta)
            


tfidf = TfidfVectorizer(binary=False,norm='l1', use_idf=True, token_pattern=r'\b[a-zA-Z]{3,}\b', stop_words=stopwords, preprocessor=stemmer)
vetor = tfidf.fit_transform(textos)
x = vetor.toarray()
y = categorias

dtree = tree.DecisionTreeClassifier(criterion='entropy')

# validação treino/teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3) 
dtree.fit(x_train, y_train)

y_predict = dtree.predict(x_test)
acuracia = accuracy_score(y_test, y_predict)
acuracia    
    

0.9642857142857143

# Cross validation

In [96]:
y_pred = cross_val_predict(dtree, x, y, cv=10)
acuracia = accuracy_score(y, y_pred)
# precisao e revocacao
pr = precision_recall_fscore_support(y, y_pred)
# matriz de confusão
cf = confusion_matrix(y,y_pred)
# mostrar matrix
classes =sorted(set(y))
print(classes)
for i, c in enumerate(cf):
    print(c)
print('precisão: {}; revocação: {}; fscore: {}; acurácia: {}'.format(pr[0], pr[1], pr[2], acuracia))
#acc

['CTI', 'agricultura', 'cultura', 'esportes']
[90  0  0  0]
[ 0 95  4  0]
[ 0  1 59  0]
[ 1  0  0 29]
precisão: [0.98901099 0.98958333 0.93650794 1.        ]; revocação: [1.         0.95959596 0.98333333 0.96666667]; fscore: [0.99447514 0.97435897 0.95934959 0.98305085]; acurácia: 0.978494623655914
